In [1]:
import os
import subprocess
from osgeo import gdal

In [24]:
import os
import numpy as np
from osgeo import gdal, osr

# Define the input and output file paths
input_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/data/masked_raster.tif'
nan_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/nan_raster.tif'
reprojected_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif'
colored_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/colored_raster.tif'
output_directory = '/Users/vivianhuang/Documents/GitHub/raster_process/final_data/'
color_table = 'precipitation_color_table.txt'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

def set_negative_to_nan(input_file, output_file, threshold=-9999):
    print(f"Setting values less than {threshold} to NaN in {input_file}")

    # Open the source dataset
    src_ds = gdal.Open(input_file, gdal.GA_ReadOnly)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")

    # Get raster band
    band = src_ds.GetRasterBand(1)
    data = band.ReadAsArray()

    # Set values below the threshold to NaN
    data = np.where(data < threshold, np.nan, data)

    # Create a new dataset with the same dimensions and projection
    driver = gdal.GetDriverByName('GTiff')
    dst_ds = driver.Create(output_file, src_ds.RasterXSize, src_ds.RasterYSize, 1, gdal.GDT_Float32)
    dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
    dst_ds.SetProjection(src_ds.GetProjection())

    # Write the modified data to the new dataset
    dst_band = dst_ds.GetRasterBand(1)
    dst_band.WriteArray(data)
    dst_band.SetNoDataValue(np.nan)

    # Flush data to disk and close datasets
    dst_band.FlushCache()
    dst_ds = None
    src_ds = None
    print(f"Values less than {threshold} set to NaN and saved to {output_file}")

def reproject_geotiff(input_file, output_file, dst_srs='EPSG:3857'):
    print(f"Reprojecting {input_file} to {output_file} with SRS {dst_srs}")
    
    # Open the source dataset
    src_ds = gdal.Open(input_file)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")
    
    # Define the target spatial reference
    dst_srs = osr.SpatialReference()
    dst_srs.ImportFromEPSG(3857)
    
    # Perform the reprojection
    warp_options = gdal.WarpOptions(
        dstSRS=dst_srs.ExportToWkt(),
        srcNodata=np.nan,
        dstNodata=np.nan,
        resampleAlg=gdal.GRA_NearestNeighbour
    )
    dst_ds = gdal.Warp(output_file, src_ds, options=warp_options)
    
    if dst_ds is None:
        raise RuntimeError(f"Reprojection failed for {input_file}")
    
    print("GeoTIFF reprojected successfully")

# def apply_color_map(input_file, color_table, output_file):
#     print(f"Applying color map from {color_table} to {input_file}")
#     command = f'gdaldem color-relief -of GTiff {input_file} {color_table} {output_file} -alpha'
#     os.system(command)
#     print(f"Color map applied and saved to {output_file}")

def generate_tiles(input_file, output_dir, zoom_levels):
    print(f"Generating tiles from {input_file} to {output_dir} for zoom levels {zoom_levels}")
    input_ds = gdal.Open(input_file)
    if input_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")

    xsize = input_ds.RasterXSize
    ysize = input_ds.RasterYSize
    print(f"Input dataset dimensions: xsize={xsize}, ysize={ysize}")

    tile_size = 256

    for zoom in range(zoom_levels[0], zoom_levels[1] + 1):
        scale = 2 ** zoom
        tile_grid_xsize = (xsize * scale + tile_size - 1) // tile_size  # Round up to ensure coverage
        tile_grid_ysize = (ysize * scale + tile_size - 1) // tile_size  # Round up to ensure coverage

        if tile_grid_xsize == 0 or tile_grid_ysize == 0:
            print(f"Skipping zoom level {zoom} due to zero-sized tile grid")
            continue

        print(f"Zoom level {zoom}: scale={scale}, tile_grid_xsize={tile_grid_xsize}, tile_grid_ysize={tile_grid_ysize}")

        for x in range(tile_grid_xsize):
            for y in range(tile_grid_ysize):
                xoff = x * tile_size // scale
                yoff = y * tile_size // scale
                win_xsize = min(tile_size // scale, xsize - xoff)
                win_ysize = min(tile_size // scale, ysize - yoff)

                tile_filename = os.path.join(output_dir, str(zoom), str(x), f"{y}.png")
                os.makedirs(os.path.dirname(tile_filename), exist_ok=True)

                print(f"Creating tile {tile_filename} from window ({xoff}, {yoff}, {win_xsize}, {win_ysize})")
                gdal.Translate(tile_filename, input_ds,
                               srcWin=[xoff, yoff, win_xsize, win_ysize],
                               width=tile_size, height=tile_size,
                               outputType=gdal.GDT_Byte,
                               format="PNG")

    print("Tiles generated successfully")

# Run the steps
set_negative_to_nan(input_geotiff, nan_geotiff, threshold=-9999)
reproject_geotiff(nan_geotiff, reprojected_geotiff)
#apply_color_map(reprojected_geotiff, color_table, colored_geotiff)
generate_tiles(reprojected_geotiff, output_directory, (0, 6))


Setting values less than -9999 to NaN in /Users/vivianhuang/Documents/GitHub/raster_process/data/masked_raster.tif
Values less than -9999 set to NaN and saved to /Users/vivianhuang/Documents/GitHub/raster_process/process_data/nan_raster.tif
Reprojecting /Users/vivianhuang/Documents/GitHub/raster_process/process_data/nan_raster.tif to /Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif with SRS EPSG:3857
GeoTIFF reprojected successfully
Generating tiles from /Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif to /Users/vivianhuang/Documents/GitHub/raster_process/final_data/ for zoom levels (0, 6)
Input dataset dimensions: xsize=60, ysize=39
Zoom level 0: scale=1, tile_grid_xsize=1, tile_grid_ysize=1
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/0/0/0.png from window (0, 0, 60, 39)
Zoom level 1: scale=2, tile_grid_xsize=1, tile_grid_ysize=1
Creating tile /Users/vivianhuang/Documents/GitHub/ras

Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.

Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/2.png from window (4, 8, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/3.png from window (4, 12, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/4.png from window (4, 16, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/5.png from window (4, 20, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/6.png from window (4, 24, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/7.png from window (4, 28, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/8.png from window (4, 32, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/1/9.png from window (4, 36, 4, 3)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/2/0.png from window (8, 0, 

Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.

Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/8/9.png from window (32, 36, 4, 3)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/0.png from window (36, 0, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/1.png from window (36, 4, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/2.png from window (36, 8, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/3.png from window (36, 12, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/4.png from window (36, 16, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/5.png from window (36, 20, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/6.png from window (36, 24, 4, 4)
Creating tile /Users/vivianhuang/Documents/GitHub/raster_process/final_data/6/9/7.png from window (

Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.
Warning 1: for band 1, nodata value has been rounded to 0, Byte being an integer datatype.

In [29]:
import os
import numpy as np
from osgeo import gdal, osr
from PIL import Image

# Define the input and output file paths
input_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/data/masked_raster.tif'
nan_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/nan_raster.tif'
reprojected_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif'
colored_geotiff = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/colored_raster.tif'
output_gif = '/Users/vivianhuang/Documents/GitHub/raster_process/process_data/output.gif'
color_table = '/Users/vivianhuang/Documents/GitHub/raster_process/precipitation_color_table.txt'  # Ensure this path is correct

def set_negative_to_nan(input_file, output_file, threshold=-9999):
    print(f"Setting values less than {threshold} to NaN in {input_file}")

    # Open the source dataset
    src_ds = gdal.Open(input_file, gdal.GA_ReadOnly)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")

    # Get raster band
    band = src_ds.GetRasterBand(1)
    data = band.ReadAsArray()

    # Set values below the threshold to NaN
    data = np.where(data < threshold, np.nan, data)

    # Create a new dataset with the same dimensions and projection
    driver = gdal.GetDriverByName('GTiff')
    dst_ds = driver.Create(output_file, src_ds.RasterXSize, src_ds.RasterYSize, 1, gdal.GDT_Float32)
    dst_ds.SetGeoTransform(src_ds.GetGeoTransform())
    dst_ds.SetProjection(src_ds.GetProjection())

    # Write the modified data to the new dataset
    dst_band = dst_ds.GetRasterBand(1)
    dst_band.WriteArray(data)
    dst_band.SetNoDataValue(np.nan)

    # Flush data to disk and close datasets
    dst_band.FlushCache()
    dst_ds = None
    src_ds = None
    print(f"Values less than {threshold} set to NaN and saved to {output_file}")

def reproject_geotiff(input_file, output_file, dst_srs='EPSG:3857'):
    print(f"Reprojecting {input_file} to {output_file} with SRS {dst_srs}")
    
    # Open the source dataset
    src_ds = gdal.Open(input_file)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")
    
    # Define the target spatial reference
    dst_srs = osr.SpatialReference()
    dst_srs.ImportFromEPSG(3857)
    
    # Perform the reprojection
    warp_options = gdal.WarpOptions(
        dstSRS=dst_srs.ExportToWkt(),
        srcNodata=np.nan,
        dstNodata=np.nan,
        resampleAlg=gdal.GRA_NearestNeighbour
    )
    dst_ds = gdal.Warp(output_file, src_ds, options=warp_options)
    
    if dst_ds is None:
        raise RuntimeError(f"Reprojection failed for {input_file}")
    
    print("GeoTIFF reprojected successfully")

def apply_color_map(input_file, color_table, output_file):
    print(f"Applying color map from {color_table} to {input_file}")
    command = f'gdaldem color-relief -of GTiff {input_file} {color_table} {output_file} -alpha'
    os.system(command)
    print(f"Color map applied and saved to {output_file}")

def generate_transparent_gif(input_file, output_file):
    print(f"Generating transparent GIF from {input_file}")

    # Open the source dataset
    src_ds = gdal.Open(input_file)
    if src_ds is None:
        raise FileNotFoundError(f"Unable to open {input_file}")

    # Get raster band
    band = src_ds.GetRasterBand(1)
    data = band.ReadAsArray()

    # Normalize data for better color representation
    min_val = np.nanmin(data)
    max_val = np.nanmax(data)
    normalized_data = (data - min_val) / (max_val - min_val) * 255
    normalized_data = np.uint8(np.where(np.isnan(normalized_data), 0, normalized_data))

    # Create an image with transparency
    img = Image.fromarray(normalized_data, mode='L')
    img = img.convert("RGBA")
    datas = img.getdata()

    new_data = []
    for item in datas:
        # Change all 0 values to transparent
        if item[0] == 0:
            new_data.append((255, 255, 255, 0))
        else:
            new_data.append(item)

    img.putdata(new_data)
    img.save(output_file, "GIF")
    print(f"GIF saved to {output_file}")

# Run the steps
set_negative_to_nan(input_geotiff, nan_geotiff, threshold=-9999)
reproject_geotiff(nan_geotiff, reprojected_geotiff)
apply_color_map(reprojected_geotiff, color_table, colored_geotiff)
generate_transparent_gif(colored_geotiff, output_gif)


Setting values less than -9999 to NaN in /Users/vivianhuang/Documents/GitHub/raster_process/data/masked_raster.tif
Values less than -9999 set to NaN and saved to /Users/vivianhuang/Documents/GitHub/raster_process/process_data/nan_raster.tif
Reprojecting /Users/vivianhuang/Documents/GitHub/raster_process/process_data/nan_raster.tif to /Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif with SRS EPSG:3857
GeoTIFF reprojected successfully
Applying color map from /Users/vivianhuang/Documents/GitHub/raster_process/precipitation_color_table.txt to /Users/vivianhuang/Documents/GitHub/raster_process/process_data/reproject_raster.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Color map applied and saved to /Users/vivianhuang/Documents/GitHub/raster_process/process_data/colored_raster.tif
Generating transparent GIF from /Users/vivianhuang/Documents/GitHub/raster_process/process_data/colored_raster.tif
GIF saved to /Users/vivianhuang/Documents/Gi